# Data munging concepts


In [0]:
from pyspark.sql.session import SparkSession#15lakhs
spark=SparkSession.builder.appName("WD37 - ETL Pipeline - Bread & Butter").getOrCreate()#3 lakhs LOC by Databricks (for eg. display, delta, xml)

In [0]:
df_raw = spark.read.options(header=False, inferSchema=True).format("csv").load("/Volumes/workspace/default/logan_datalake/customers_raw_more_rem_head.txt").toDF("id","fname","lname","age","location","proffesion")
#display(df_raw.limit(10))
display(df_raw.sample(.1))
#df_raw.count()
df_raw.printSchema()

In [0]:
#from pyspark.sql.functions import col,trim
#display(df_raw.filter(df_raw.fname.isNull() | df_raw.lname.isNull())) # dsl
#df_fnull = df_raw.where(df_raw.fname.isNull() | df_raw.lname.isNull()) # sel
df_rm_null_names = df_raw.where(df_raw.fname.isNotNull() & df_raw.lname.isNotNull())
#display(df_fnull)

In [0]:
print("actual count of the data",df_raw.count())
print("distinct on orginal cnt",df_raw.distinct().count())
print("distinct on id",df_raw.select('id').distinct().count())
print("fname,lname null removed",df_rm_null_names.count())
print("Dropping duplicates by id",df_raw.dropDuplicates(['id']).count())
print("Dropping duplicates in df",df_raw.dropDuplicates().count())
display(df_raw.describe())
display(df_raw.summary())

#Extraction (Ingestion) methodologies


In [0]:
header = "id string,fname string,lname string,age string,location string,proffesion string"
#single file different header calling method
##------------------------------------Calling multiple files-----------------------
#raw_df = spark.read.format("csv").load("/Volumes/workspace/default/logan_datalake/customers_raw_more_rem_head.txt").toDF(*header.split(","))
##---------
#raw_df = spark.read.schema(header).format("csv").load("/Volumes/workspace/default/logan_datalake/customers_raw_more_rem_head.txt",inferSchema=True,header=False)
#------------------------------------Calling multiple files-----------------------
raw_df = spark.read.schema(header).csv(path="/Volumes/workspace/default/logan_datalake/customers_raw_more_rem_head.txt",inferSchema=True,header=False)
raw_df1 = spark.read.schema(header).csv(path="/Volumes/workspace/default/logan_datalake/customers_raw_more.txt",inferSchema=True,header=False)
raw_df2 = spark.read.schema(header).csv(path="/Volumes/workspace/default/logan_datalake/customers_raw_miss_col.txt",inferSchema=True,header=False)
#,recursiveFileLookup=True,pathGlobFilter="custsm*") this is used for multiple files in different folders
#raw_df = spark.read.schema(header).csv(path=["/Volumes/workspace/default/logan_datalake/customers_raw_more_rem_head.txt","/Volumes/workspace/default/logan_datalake/customers_raw_more.txt"],inferSchema=True,header=False)

print('raw data cnt',raw_df.count(),raw_df1.count(),raw_df2.count())

#Active Data munging...

In [0]:
#union/merged
#df_union = raw_df.union(raw_df1)
df_union1 = raw_df.unionByName(raw_df2,allowMissingColumns=True)#unionByName is used for different columns
print('union data cnt',df_union1.count())
#display(df_union1)


In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,NumericType
#Validation by doing cleansing line 178
#mode="permissive" (DEFAULT)
        #Allow bad rows, do not fail, do not drop
        #Put invalid/broken row content into a special column
#mode='dropMalformed'
        #no corrupt column, no nulls — Spark removes the entire row
        #If a row doesn't match schema → it is thrown away

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema_df = StructType([
    StructField("id", IntegerType(), True),
    StructField("fname", StringType(), True),
    StructField("lname", StringType(), True),
    StructField("age", DoubleType(), True),
    StructField("location", StringType(), True),
    StructField("proffesion", StringType(), True)
])
#mode="permissive"
df_modes_permissive = spark.read.schema(schema_df).csv(
    path="/Volumes/workspace/default/logan_datalake/customers_raw_more_rem_head.txt",
    header=False,columnNameOfCorruptRecord="corrupt_record"
)

df_modes_dropMalformed = spark.read.schema(schema_df).csv(
    path="/Volumes/workspace/default/logan_datalake/customers_raw_more_rem_head.txt",
    header=False,mode="dropMalformed"
)

print("Permissive mode cnt",df_modes_permissive.count())
print("DropMalformed mode cnt",df_modes_dropMalformed.count())
print("after cleaning wrong data (type mismatch, column number mismatch)",len(df_modes_dropMalformed.collect()))

 
#####Rejection Strategy


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema_df = StructType([
    StructField("id", IntegerType(), True),
    StructField("fname", StringType(), True),
    StructField("lname", StringType(), True),
    StructField("age", DoubleType(), True),
    StructField("location", StringType(), True),
    StructField("proffesion", StringType(), True),
    StructField("corrupt_record", StringType(), True)
])

df_modes_permissive = spark.read.schema(schema_df).csv(
    path="/Volumes/workspace/default/logan_datalake/customers_raw_more_rem_head.txt",
    header=False,columnNameOfCorruptRecord="corrupt_record"
)
#display(df_modes_permissive)
#display(df_modes_permissive.where(df_modes_permissive.corrupt_record.isNull()))#cleaned rows

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
#Important na functions we can use to do cleansing.
schema_df = StructType([
    StructField("id", IntegerType(), True),
    StructField("fname", StringType(), True),
    StructField("lname", StringType(), True),
    StructField("age", DoubleType(), True),
    StructField("location", StringType(), True),
    StructField("proffesion", StringType(), True)])

df_raw = spark.read.schema(schema_df).csv(
    path="/Volumes/workspace/default/logan_datalake/customers_raw_more_rem_head.txt",
    header=False)
#This function will drop any column in a given row with null otherwise this function returns rows with no null columns - In a scenario of if the source send the Datascience Model features (we shouldn't have any one feature with null value, hence we can use this function)
cleanseddf=df_raw.na.drop(how="any")
cleanseddf1=df_raw.na.drop(how="any",subset=["id","age"])
#display(df_raw)
#display(cleanseddf1)
#display(cleanseddf)

#####Scrubbing 
na.fill() & na.replace()


In [0]:

#df_fill = df_raw.na.fill({'id': 0, 'age': 0,'fname': 'missing','lname': 'missing','proffesion': 'Not'})  
df_fill=df_raw.na.fill('not provided',subset=["lastname","profession"])# this will replace null with given val
#df_replace = df_raw.na.replace(['chennai'], ['che'], 'location') 
find_replace_values_dict1={'chennai':'Chennai','delhi':'Head off'}
df_replace = df_raw.na.replace(find_replace_values_dict1,subset=['location'])
 
#display(df_fill)
#display(df_replace)

#####DeDuplication
Removal of duplicate rows/columns based on a priority or non priority
distinct & dropDuplicates



In [0]:
display(df_raw.select("id").groupBy("id").count().filter("count > 1"))

display(df_raw.select("*").where("id = 1010 or id is null"))
dedupdf2=df_raw.dropDuplicates(subset=["id"])

display(dedupdf2.select("*").where("id = 1010 or id is null"))

display(df_raw.dropDuplicates(subset=["id"]).select("*").where("id = 1010 or id is null"))